### 1.1 线性探查闭散列下的哈希表
#### 一. 惰性删除
1. 哈希表往往采用闭散列策略处理冲突.  
2. 闭散列的策略通常为为线性试探,所以相同hash值的key会集中在哈希表的某一段.  
 由于这种局部性, 导致在删除节点时产生空桶二破坏这种局部性. 因此可谓每个桶设置一个标记, 指示该桶虽然目前为空, 但此前曾存放过词条
 
#### 二. 哈希表查找  
1. 在线性探查下, 查找命中位置.   
 有两种试探终止的可能`(__probe4hit(self,key))`:   
   1. 在一个非空桶内查找到关键码   
   2. 线性探查到一个懒删除标记为0的空桶(表示该桶的元素从未被删除过)  
2. 哈希表的查找首先使用`__probe4hit(self,key)`查找命中位置, 若该位置上有元素, 则查找成功; 若该位置上桶为空, 则查找失败

#### 三. 哈希表插入
1. 哈希表插入需要先找到可插入的位置  
 只要可插入位置上没有元素, 就插入. 这个插入与lazy_removal标记无关  
2. 为什么线性探查下, 总能找到能插入的空桶, 而不会出现所有桶都有元素的情况 ?   
 因为哈希表的装填因子只要不是100%, 就一定会留下空桶. 所以装填因子在哈希表的处理上至关重要.  
 只要哈希表的装填因子大于50%, 就要进行二次哈希  
3. 二次哈希保证装填因子小于50%
 当发现元素数量到达表长的一半时, 说明装填因子已经达到50%, 此时应该进行二次哈希  
 二次哈希的步骤为将元素整体搬迁到扩容后的新散列表下, 其步骤为将原来的元素依次`put`到新的哈希表下
 

In [15]:
import numpy as np

class Entry(object):
    def __init__(self,k,v):
        self.key = k
        self.value = v
    def __str__(self):
        return '%s->%s'%(self.key,self.value)

class HashTable(object):
    def __init__(self,bucket_number=5):
        '''bucket_number:哈希表长,也是桶的数量
           entry_number:词条数量'''
        self.M = bucket_number
        self.N = 0  # 词条数量
        self.ht = np.array([None for i in range(bucket_number)],dtype=object) # key数组
        self.lazy_removal = np.array([0 for i in range(bucket_number)])       # 懒删除标记

    def __hashcode(self,k):
        '''哈希函数: 计算哈希值有多种方法:如强制转换整数,对象成员求和,多项式散列码
           此处仅以强制转换整数为例'''
        return int(k)

    def __probe4hit(self,key):
        '''探查命中位置:
           此处以线性探查为例子(惰性删除下的查找策略)'''
        r = self.__hashcode(key) % self.M  # 根据key的哈希值计算桶的位置
        while (self.ht[r] is not None and key!=self.ht[r].key) or (self.ht[r] is None and self.lazy_removal[r]==1):
            print r
            r = (r+1) % self.M
        return r

    def get(self,k):
        r = self.__probe4hit(k)
        return None if self.ht[r] is None else self.ht[r].value

    def remove(self,key):
        r = self.__probe4hit(k)
        if ht[r].key is None:
            raise RuntimeError('not key in this hashtable')
        else:
            ht[r]=None
            self.N = self.N-1
            self.lazy_removal[r] = 1
            return True

    def __probe4Free(self,key):
        r = self.__hashcode(key) % self.M
        while self.ht[r] is not None:
            r = (r+1)%self.M
        return r

    def __rehash(self):
        '''二次哈希'''
        old_capacity = self.M
        old_ht = self.ht
        self.M = old_capacity * 2 # 简单的进行2倍扩容,应该是在找到2*M后的一个素数
        self.N = 0
        self.ht = np.array([None for i in range(self.M)],dtype=object) # key数组
        self.lazy_removal = np.array([0 for i in range(self.M)]) # 懒删除标记
        for i in range(old_capacity):
            if old_ht[i] is not None:
                self.put(old_ht[i].key,old_ht[i].value)


    def put(self,k,v):
        if self.ht[self.__probe4hit(k)] is not None:
            raise RuntimeError('don\'t insert same key')
        r = self.__probe4Free(k)
        self.ht[r] = Entry(k,v)
        self.N = self.N + 1
        if self.N*2 > self.M:
            self.__rehash()
        return
    
    def __str__(self):
        return [entry.__str__() for entry in self.ht].__str__()

In [16]:
if __name__ == '__main__':
    ht = HashTable()
    ht.put(1,'a')
    ht.put(2,'b')
    ht.put(3,'c') # 触发rehash
    print 'key:1,value:',ht.get(1)
    print 'hash table:',ht

key:1,value: a
hash table: ['None', '1->a', '2->b', '3->c', 'None', 'None', 'None', 'None', 'None', 'None']


### 1.2 其他三列策略和哈希计算函数